# 04 - Price Statistical Models

## Objective
Apply classical time series statistical models to price forecasting.

**Models:**
1. SARIMA (Seasonal AutoRegressive Integrated Moving Average)
2. ETS (Exponential Smoothing State Space Model)

**Hypothesis:**
- Statistical models may struggle with price volatility
- SARIMA may work better if seasonality is strong
- Expect R² lower than solar/wind due to price spikes

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Statistical models
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Data

In [ ]:
# Load original (non-scaled) data for statistical models
from pathlib import Path
data_path = Path('../../data/raw/price_day_ahead_2022-01-01_2024-12-31_hour.csv')
df = pd.read_csv(data_path)
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)
df.columns = ['price']

# Split
val_start = '2024-07-01'
test_start = '2024-10-01'

train = df[:val_start].copy()
val = df[val_start:test_start].copy()
test = df[test_start:].copy()

y_train = train['price']
y_val = val['price']
y_test = test['price']

print(f"Train: {len(y_train)} hours")
print(f"Val:   {len(y_val)} hours")
print(f"Test:  {len(y_test)} hours")

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """Calculate evaluation metrics"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true.abs() + 1e-8))) * 100
    
    return {
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE': mape
    }

## 2. Stationarity Test

In [ ]:
# Augmented Dickey-Fuller test
result = adfuller(y_train.dropna())

print("Augmented Dickey-Fuller Test:")
print(f"  ADF Statistic: {result[0]:.4f}")
print(f"  p-value: {result[1]:.4f}")
print(f"  Critical Values:")
for key, value in result[4].items():
    print(f"    {key}: {value:.3f}")

if result[1] < 0.05:
    print("\n✅ Series is STATIONARY (p < 0.05)")
else:
    print("\n⚠️ Series is NON-STATIONARY (p >= 0.05) - differencing needed")

## 3. SARIMA Model

In [ ]:
# SARIMA - Manual order (based on ACF/PACF from exploration)
# Using seasonal order (24 hours)
print("Training SARIMA model...")
print("This may take several minutes...\n")

# Start with simple order
order = (1, 1, 1)  # (p, d, q)
seasonal_order = (1, 1, 1, 24)  # (P, D, Q, s) - 24 hour seasonality

try:
    sarima_model = SARIMAX(y_train, 
                           order=order,
                           seasonal_order=seasonal_order,
                           enforce_stationarity=False,
                           enforce_invertibility=False)
    
    sarima_fit = sarima_model.fit(disp=False, maxiter=200)
    
    print(f"✅ SARIMA{order}×{seasonal_order} trained successfully")
    print(f"   AIC: {sarima_fit.aic:.2f}")
    print(f"   BIC: {sarima_fit.bic:.2f}")
    
except Exception as e:
    print(f"❌ SARIMA training failed: {e}")
    sarima_fit = None

In [ ]:
# SARIMA predictions
if sarima_fit is not None:
    print("Generating SARIMA forecast...")
    
    # Forecast test period
    sarima_forecast = sarima_fit.forecast(steps=len(y_test))
    sarima_results = evaluate_model(y_test, sarima_forecast, 'SARIMA(1,1,1)(1,1,1,24)')
    
    print("\nSARIMA Results:")
    print(f"  R²: {sarima_results['R²']:.4f}")
    print(f"  RMSE: {sarima_results['RMSE']:.2f}")
    print(f"  MAE: {sarima_results['MAE']:.2f}")
else:
    print("Skipping SARIMA evaluation (training failed)")
    sarima_results = None
    sarima_forecast = None

## 4. ETS (Exponential Smoothing)

In [ ]:
# ETS model with additive trend and seasonal components
print("Training ETS model...")

try:
    ets_model = ExponentialSmoothing(
        y_train,
        seasonal_periods=24,  # hourly seasonality
        trend='add',
        seasonal='add',
        use_boxcox=False
    )
    
    ets_fit = ets_model.fit()
    print("✅ ETS model trained successfully")
    
except Exception as e:
    print(f"❌ ETS training failed: {e}")
    ets_fit = None

In [ ]:
# ETS predictions
if ets_fit is not None:
    print("Generating ETS forecast...")
    
    ets_forecast = ets_fit.forecast(steps=len(y_test))
    ets_results = evaluate_model(y_test, ets_forecast, 'ETS (A,A,A)')
    
    print("\nETS Results:")
    print(f"  R²: {ets_results['R²']:.4f}")
    print(f"  RMSE: {ets_results['RMSE']:.2f}")
    print(f"  MAE: {ets_results['MAE']:.2f}")
else:
    print("Skipping ETS evaluation (training failed)")
    ets_results = None
    ets_forecast = None

## 5. Results Comparison

In [ ]:
# Load baseline results for comparison
baseline_df = pd.read_csv('../../results/metrics/price_baseline_metrics.csv')

# Combine results
results_list = baseline_df.to_dict('records')

if sarima_results:
    results_list.append(sarima_results)
if ets_results:
    results_list.append(ets_results)

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('R²', ascending=False)

print("\n" + "="*80)
print("STATISTICAL MODELS vs BASELINES")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² comparison
colors = ['steelblue' if 'SARIMA' in m or 'ETS' in m else 'lightgray' 
          for m in results_df['Model']]
axes[0].barh(results_df['Model'], results_df['R²'], color=colors, edgecolor='black')
axes[0].set_xlabel('R² Score')
axes[0].set_title('R² Score Comparison', fontweight='bold')
axes[0].grid(alpha=0.3, axis='x')

# RMSE comparison
axes[1].barh(results_df['Model'], results_df['RMSE'], color=colors, edgecolor='black')
axes[1].set_xlabel('RMSE')
axes[1].set_title('RMSE Comparison', fontweight='bold')
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../../results/figures/price_statistical_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Forecast Visualization

In [ ]:
# Plot forecasts (first 7 days)
plot_days = 7
plot_hours = plot_days * 24

fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(y_test.index[:plot_hours], y_test.values[:plot_hours], 
        linewidth=2.5, label='Actual', color='black', zorder=5)

if sarima_forecast is not None:
    ax.plot(y_test.index[:plot_hours], sarima_forecast[:plot_hours], 
            linewidth=1.5, label='SARIMA', alpha=0.7, linestyle='--')

if ets_forecast is not None:
    ax.plot(y_test.index[:plot_hours], ets_forecast[:plot_hours], 
            linewidth=1.5, label='ETS', alpha=0.7, linestyle='--')

ax.axhline(0, color='red', linestyle='-', linewidth=1)
ax.set_title(f'Statistical Models Forecast - First {plot_days} Days', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.legend(loc='best')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../results/figures/price_statistical_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Residual Analysis

In [ ]:
if sarima_fit is not None:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Residuals plot
    residuals = sarima_fit.resid
    
    # Time series plot
    axes[0, 0].plot(residuals)
    axes[0, 0].axhline(0, color='red', linestyle='--', linewidth=1)
    axes[0, 0].set_title('SARIMA Residuals Over Time', fontweight='bold')
    axes[0, 0].set_ylabel('Residual')
    axes[0, 0].grid(alpha=0.3)
    
    # Histogram
    axes[0, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[0, 1].set_title('Residuals Distribution', fontweight='bold')
    axes[0, 1].set_xlabel('Residual')
    axes[0, 1].grid(alpha=0.3)
    
    # ACF of residuals
    from statsmodels.graphics.tsaplots import plot_acf
    plot_acf(residuals.dropna(), lags=48, ax=axes[1, 0])
    axes[1, 0].set_title('ACF of Residuals', fontweight='bold')
    
    # Q-Q plot
    from scipy import stats
    stats.probplot(residuals.dropna(), dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title('Q-Q Plot', fontweight='bold')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../../results/figures/price_sarima_residuals.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("Skipping residual analysis (SARIMA not available)")

## 8. Save Results

In [ ]:
# Save statistical models metrics
statistical_results = []
if sarima_results:
    statistical_results.append(sarima_results)
if ets_results:
    statistical_results.append(ets_results)

if statistical_results:
    statistical_df = pd.DataFrame(statistical_results)
    statistical_df.to_csv('../../results/metrics/price_statistical_metrics.csv', index=False)
    print("✅ Statistical models results saved")
else:
    print("⚠️ No statistical models to save")

## 9. Summary

In [ ]:
print("="*80)
print("📋 PRICE STATISTICAL MODELS - SUMMARY")
print("="*80)

if sarima_results:
    print(f"\n🔹 SARIMA:")
    print(f"   R²: {sarima_results['R²']:.4f}")
    print(f"   RMSE: {sarima_results['RMSE']:.2f}")
    print(f"   MAE: {sarima_results['MAE']:.2f}")

if ets_results:
    print(f"\n🔹 ETS:")
    print(f"   R²: {ets_results['R²']:.4f}")
    print(f"   RMSE: {ets_results['RMSE']:.2f}")
    print(f"   MAE: {ets_results['MAE']:.2f}")

print(f"\n🏆 BEST STATISTICAL MODEL:")
if sarima_results and ets_results:
    if sarima_results['R²'] > ets_results['R²']:
        print(f"   SARIMA (R²={sarima_results['R²']:.4f})")
    else:
        print(f"   ETS (R²={ets_results['R²']:.4f})")
elif sarima_results:
    print(f"   SARIMA (R²={sarima_results['R²']:.4f})")
elif ets_results:
    print(f"   ETS (R²={ets_results['R²']:.4f})")

best_baseline = results_df.iloc[0]
print(f"\n📊 vs BEST BASELINE ({best_baseline['Model']}):")
print(f"   Baseline R²: {best_baseline['R²']:.4f}")

print(f"\n💡 INSIGHTS:")
print(f"   - Price volatility makes statistical models challenging")
print(f"   - Negative prices and spikes affect performance")
print(f"   - ML models (next) expected to perform better")

print("\n" + "="*80)
print("✅ Statistical models complete! Ready for ML tree models.")
print("="*80)

## Next Steps

1. ✅ Data exploration
2. ✅ Data preprocessing
3. ✅ Baseline models
4. ✅ Statistical models
5. ➡️ **Next:** `05_price_ml_tree_models.ipynb`
   - Random Forest, XGBoost, LightGBM, CatBoost
   - Feature importance analysis
   - Expected: Higher R² than statistical models
6. 📊 Then: Deep learning models